In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

In [3]:
import tensorflow as tf

In [4]:
from tensorflow.keras.applications import ResNet50

In [5]:
from tensorflow.keras import Model, Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, Flatten, Input, Concatenate
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import LearningRateScheduler

In [6]:
import numpy as np
from sklearn import metrics

In [7]:
from data.pascalvoc.pascalvoc import PascalVOC

Using TensorFlow backend.


In [8]:
from model.networks.baseline import Baseline
from model.networks.prior import PriorModel

In [9]:
from model.callbacks.metric_callbacks import MAPCallback
from model.callbacks.save_callback import SaveModel
from model.callbacks.scheduler import lr_scheduler

In [10]:
from model.losses import BCE, PartialBCE

In [11]:
from sklearn.metrics import average_precision_score

In [12]:
from config import config_utils

In [13]:
from config.config import cfg

In [14]:
prop=100

In [ ]:
parse_options_file('/home/caleml/partial-labels/config/pv_baseline50_sgd_448lrs.yaml')

## model

In [ ]:
# From http://cedric.cnam.fr/vertigo/Cours/ml2/tpDeepLearning5.html

In [ ]:
# Load ResNet50 architecture & its weights
# imagenet_model = ResNet50(include_top=True, weights='imagenet')
# model.layers.pop()
# Modify top layers
# x = model.layers[-1].output
# x = Dense(data_generator_train.nb_classes, activation='sigmoid', name='predictions')(x)
# model = Model(inputs=model.input, outputs=x)

In [ ]:
# imagenet only model (no finetuning)
model = ResNet50(include_top=True, weights='imagenet')

In [ ]:
# model.layers.pop().pop()
model = Model(inputs=model.input,outputs=model.layers[-2].output)

In [ ]:
model.summary()

In [ ]:
lr = 0.1
model.compile(loss='binary_crossentropy', optimizer=SGD(lr, momentum=0.9), metrics=['binary_accuracy'])

## feature extraction

In [ ]:
data_dir = '/share/DEEPLEARNING/datasets/pascalvoc/VOCdevkit/VOC2007/'
data_generator_train = PascalVOCDataGenerator('trainval', data_dir, prop=prop)

In [ ]:
data_generator_train = PascalVOCDataGenerator('trainval', data_dir, force_old=True)

In [ ]:
batch_size = 32
generator = data_generator_train.flow(batch_size=batch_size)

In [ ]:
# Initilisation des matrices contenant les Deep Features et les labels
X_train = np.zeros((len(data_generator_train.images_ids_in_subset),2048))
Y_train = np.zeros((len(data_generator_train.images_ids_in_subset),20))

# Calcul du nombre e batchs
nb_batches = int(len(data_generator_train.images_ids_in_subset) / batch_size) + 1

for i in range(nb_batches):
    # Pour chaque batch, on extrait les images d'entrée X et les labels y
    X, y = next(generator)
    # On récupère les Deep Feature par appel à predict
    y_pred = model.predict(X)
    X_train[i*batch_size:(i+1)*batch_size,:] = y_pred
    Y_train[i*batch_size:(i+1)*batch_size,:] = y

In [ ]:
# same for test
data_generator_test = PascalVOCDataGenerator('test', data_dir)
generator_test = data_generator_test.flow(batch_size=batch_size)

X_test = np.zeros((len(data_generator_test.images_ids_in_subset),2048))
Y_test = np.zeros((len(data_generator_test.images_ids_in_subset),20))

nb_batches = int(len(data_generator_test.images_ids_in_subset) / batch_size) + 1

for i in range(nb_batches):
    X, y = next(generator_test)
    y_pred = model.predict(X)
    X_test[i*batch_size:(i+1)*batch_size,:] = y_pred
    Y_test[i*batch_size:(i+1)*batch_size,:] = y

In [ ]:
outfile = 'DF_ResNet50_VOC2007_test28'
np.savez(outfile, X_train=X_train, Y_train=Y_train,X_test=X_test, Y_test=Y_test)

## Train classifier only

In [ ]:
outfile = 'DF_ResNet50_VOC2007_test28.npz'
learning_rate = 0.1
nb_epoch = 20
batch_size = 32

npzfile = np.load(outfile)

X_train = npzfile['X_train']
Y_train = npzfile['Y_train']

X_test = npzfile['X_test']
Y_test = npzfile['Y_test']

print("data \n X_train=", X_train.shape, "Y_train=", Y_train.shape, " X_test=", X_test.shape, "Y_train=", Y_test.shape)

In [ ]:
model = Sequential()
model.add(Dense(20, input_dim=2048, name='fc1', activation='sigmoid'))
model.summary()


sgd = SGD(learning_rate)
model.compile(loss='binary_crossentropy',optimizer=sgd,metrics=['binary_accuracy'])

model.fit(X_train, Y_train,batch_size=batch_size, epochs=nb_epoch,verbose=1)

In [ ]:
scores = model.evaluate(X_test, Y_test, verbose=0)
print("%s TEST: %.2f%%" % (model.metrics_names[0], scores[0]*100))
print("%s TEST: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
y_pred_test = model.predict(X_test)
y_pred_train = model.predict(X_train)
AP_train = np.zeros(20)
AP_test = np.zeros(20)

for c in range(20):
    AP_train[c] = average_precision_score(Y_train[:, c], y_pred_train[:, c])
    AP_test[c] = average_precision_score(Y_test[:, c], y_pred_test[:, c])

print("MAP TRAIN =%.2f", AP_train.mean()*100)
print("MAP TEST =%.2f", AP_test.mean()*100)

## Eval

In [ ]:
data_generator_test = PascalVOCDataGenerator('test', data_dir)
len(data_generator_test.images_ids_in_subset)

In [ ]:
batch_size = len(data_generator_test.images_ids_in_subset)
generator_test = data_generator_test.flow(batch_size=batch_size)

In [ ]:
X_test, Y_test = next(generator_test)

In [ ]:
print(X_test.shape, Y_test.shape)

In [ ]:
def eval_fn(model, X_test, Y_test):
    y_pred_test = model.predict(X_test)
    #y_pred_train = model.predict(X_train)

    #AP_train = np.zeros(20)
    AP_test = np.zeros(20)
    for c in range(20):
        #AP_train[c] = average_precision_score(Y_train[:, c], y_pred_train[:, c])
        AP_test[c] = metrics.average_precision_score(Y_test[:, c], y_pred_test[:, c])

    #print "MAP TRAIN =", AP_train.mean()*100
    print("MAP TEST =", AP_test.mean()*100)
    print(AP_test)

In [ ]:
eval_fn(model, X_test, Y_test)

In [ ]:
print(AP_test)

## Finetune

In [ ]:
data_dir = '/share/DEEPLEARNING/datasets/pascalvoc/VOCdevkit/VOC2007/'
data_generator_train = PascalVOCDataGenerator('trainval', data_dir, prop=prop)

In [ ]:
# vanilla from RCP209
model = ResNet50(include_top=True, weights='imagenet')
model.layers.pop()
# Modify top layers
x = model.layers[-1].output
x = Dense(data_generator_train.nb_classes, activation='sigmoid', name='predictions')(x)
model = Model(inputs=model.input, outputs=x)

In [ ]:
# dropping the Dense(1000)
model = ResNet50(include_top=True, weights='imagenet')
x = model.layers[-2].output
x = Dense(data_generator_train.nb_classes, activation='sigmoid', name='predictions')(x)
model = Model(inputs=model.input, outputs=x)

In [ ]:
# Laura way
# Load ResNet50 architecture & its weights
input_shape = (224, 224, 3)
resnet = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)

inp = Input(shape=input_shape, name='image_input')
x = resnet(inp)
x = Flatten()(x)
output = Dense(data_generator_train.nb_classes, activation='sigmoid')(x)
model = Model(inputs=inp, outputs=output)

In [ ]:
model.summary()

In [ ]:
lr = 0.1
# model.compile(loss='binary_crossentropy', optimizer=SGD(lr=lr), metrics=['binary_accuracy'])
loss = BCE()
# loss = PartialBCE(prop / 100)
model.compile(loss=loss, optimizer=SGD(lr=0.01), metrics=['binary_accuracy'])

In [ ]:
# test data
data_generator_test = PascalVOCDataGenerator('test', data_dir)
len(data_generator_test.images_ids_in_subset)
batch_size = len(data_generator_test.images_ids_in_subset)
generator_test = data_generator_test.flow(batch_size=batch_size)
X_test, Y_test = next(generator_test)
print(X_test.shape, Y_test.shape)

In [ ]:
# callbacks
exp_folder = '/home/caleml/partial_experiments/exp_20190724_1659_TESTNB'
os.makedirs(exp_folder, exist_ok=True)

cb_list = list()
cb_list.append(SaveModel(exp_folder, prop))

data_generator_test = PascalVOCDataGenerator('test', data_dir)
map_cb = MAPCallback(X_test, Y_test, exp_folder, prop)
cb_list.append(map_cb)

# cb_list.append(LearningRateScheduler(lr_scheduler))

In [ ]:
batch_size=32
nb_epochs=25

steps_per_epoch_train = int(len(data_generator_train.id_to_label) / batch_size) + 1
model.fit_generator(data_generator_train.flow(batch_size=batch_size),
                    steps_per_epoch=steps_per_epoch_train,
                    epochs=nb_epochs,
                    callbacks=cb_list,
                    verbose=1)

## Dataset comparison

In [ ]:
data_dir = '/share/DEEPLEARNING/datasets/pascalvoc/VOCdevkit/VOC2007/'
data_generator_train = PascalVOCDataGenerator('trainval', data_dir)

In [ ]:
nico_data = data_generator_train.id_to_label
print(len(nico_data))
print(nico_data['000131'])

In [ ]:
trainval_path = '/share/DEEPLEARNING/datasets/pascalvoc/VOCdevkit/VOC2007/Annotations/annotations_multilabel_trainval.csv'

In [ ]:
laura_data = dict()
with open(trainval_path, 'r') as f_in:
    for line in f_in:
        parts = line.strip().split(',')
        laura_data[parts[0]] = [int(elt) for elt in parts[1:]]

In [ ]:
print(len(laura_data))
print(laura_data['000131'])

In [ ]:
for id_img, labels in nico_data.items():
    laura_labels = laura_data[id_img]
    converted = [l if l in [0, 1] else 0 for l in laura_labels]
    assert all([converted[i] == labels[i] for i in range(len(labels))])

## Test validation

## Test prior

In [ ]:
from model.networks.prior import PriorModel
from model.callbacks.prior_callback import PriorCallback

import tensorflow as tf

In [ ]:
exp_folder = '/home/caleml/partial_experiments/exp_20190812_1654_TESTNB'
os.makedirs(exp_folder, exist_ok=True)

In [ ]:
# dataset
data_dir = '/share/DEEPLEARNING/datasets/pascalvoc/VOCdevkit/VOC2007/'
dataset_train = PascalVOCDataGenerator('trainval', data_dir, prop=prop)

In [ ]:
model = PriorModel(exp_folder, dataset_train.nb_classes, prop)
model.load_config('pv_prior50_sgd')
model.build()

In [ ]:
# callbacks
prior_cb = PriorCallback()
fetches = [tf.assign(prior_cb.var_y_true, model.targets[0], validate_shape=False),
           tf.assign(prior_cb.var_y_pred, model.outputs[0], validate_shape=False)]
model._function_kwargs = {'fetches': fetches}


In [ ]:
# train
steps_per_epoch = int(len(dataset_train.id_to_label) / cfg.BATCH_SIZE) + 1
model.train(dataset_train.flow(batch_size=cfg.BATCH_SIZE), steps_per_epoch=steps_per_epoch, cb_list=cb_list)

## Dataset test

In [ ]:
import matplotlib.image as mpimg
from matplotlib import pyplot as plt

In [ ]:
data_dir = '/home/caleml/datasets/pascalvoc/VOCdevkit/VOC2007/'

In [ ]:
old_dataset = PascalVOCDataGenerator('trainval', data_dir, prop=prop)
new_dataset = PascalVOC(data_dir, 16, 'trainval', x_keys=['image'], y_keys=['multilabel'], p=prop)

In [ ]:
# comparison function
def compare(d_old, d_new):
    
    batch1 = next(d_old)
    batch2 = d_new[0]
    print(type(batch1), type(batch2))
    print(len(batch1), len(batch2))
    x1, y1 = batch1
    x2, y2 = batch2
    print(type(x1), type(x2))
    print(type(y1), type(y2))
    print(x1.shape, np.array(x2).shape, len(x2))
    print(y1.shape, np.array(y2).shape, len(y2))
    
    fig = plt.figure()
    a = fig.add_subplot(1,3,1)
    imgplot = plt.imshow(x1[0])
    a = fig.add_subplot(1,3,2)
    imgplot = plt.imshow(x2[0][0])
    a = fig.add_subplot(1,3,3)
    first_img = '/share/DEEPLEARNING/datasets/pascalvoc/VOCdevkit/VOC2007/JPEGImages/000005.jpg'
    img = mpimg.imread(first_img)
    imgplot = plt.imshow(img)
    
    assert np.array_equal(x1, np.array(x2[0]))
    assert np.array_equal(y1, np.array(y2[0]))

In [ ]:
compare(old_dataset.flow(batch_size=16), new_dataset)

In [ ]:
a = [1, 3, 4, 7, 8, 9]
a[[1,2,4]]

In [ ]:
a = np.zeros(6)
a[1] = 6
a[2] = 7
a[-1] = 12
a[0] = 3

b = a / sum(a)

In [ ]:
a

In [ ]:
print(b, sum(b))

## New finetune with prior

In [32]:
data_dir = '/home/caleml/datasets/pascalvoc/VOCdevkit/VOC2007/'
dataset_train = PascalVOC(data_dir, 16, 'trainval', x_keys=['image'], y_keys=['multilabel'], p=prop)

In [33]:
exp_folder = '/home/caleml/partial_experiments/exp_20190829_1434_TESTNB'
os.makedirs(exp_folder, exist_ok=True)

In [34]:
from model.networks.prior import PriorModel
model = PriorModel(exp_folder, dataset_train.nb_classes, prop)
model.load_config('pv_baseline50_sgd')
model.build()

Init input_shape (448, 448, 3)
Loading options

Loaded config

{'archi': {'classifier': 'resnet50', 'loss': 'bce', 'name': 'baseline'},
 'batch_size': 32,
 'dataset': {'name': 'pascalvoc',
             'path': '/home/caleml/datasets/pascalvoc/VOCdevkit/VOC2007/',
             'test': 'test',
             'train': 'trainval'},
 'training': {'n_epochs': 15, 'optimizer': 'sgd', 'start_lr': 0.1}}



/home/caleml/.local/lib/python3.4/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


logits shape: (?, 20) and type <class 'tensorflow.python.framework.ops.Tensor'>
inside KL2 logits size (20,)
inside KL2 cooc line size (20,)
dtype p float32, dtype q float32
prior shape Tensor("prior_3/map/TensorArrayStack/TensorArrayGatherV3:0", shape=(?, 20), dtype=float32)
Outputs shape (None, 20)
Final model summary
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image_input (InputLayer)        (None, 448, 448, 3)  0                                            
__________________________________________________________________________________________________
cls_model (Model)               (None, 14, 14, 2048) 23587712    image_input[0][0]                
__________________________________________________________________________________________________
global_average_pooling2d_3 (Glo (None, 2048)         0           cls_model[1][0]    

In [26]:
a = dataset_train[1]

In [27]:
dataset_test = PascalVOC(data_dir, 4952, 'test', x_keys=['image'], y_keys=['multilabel'])
X_test, Y_test = dataset_test[0]

In [36]:
# callbacks
cb_list = list()

# map_cb = MAPCallback(X_test, Y_test, exp_folder, prop)
cb_list.append(map_cb)

cb_list.append(SaveModel(exp_folder, prop))

In [38]:
# actual train
steps_per_epoch = len(dataset_train)
model.train(dataset_train, steps_per_epoch=steps_per_epoch, cb_list=cb_list)

Training with 2 callbacks
Epoch 1/15
313/314 [============================>.] - ETA: 0s - loss: 14.7726 - binary_accuracy: 0.0060there is Tensor("count_nonzero_1/Sum:0", shape=(), dtype=int64) nans


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

## debug

In [ ]:
x = np.arange(9).reshape(3,3)

In [ ]:
print(x)
print(x.shape)

In [ ]:
x1 = np.repeat(x[None,...],10,0)

In [ ]:
x1

In [ ]:
x1.shape

In [ ]:
x1[0]

In [ ]:
from tensorflow.keras import backend as K

In [ ]:
def kullback_leibler_div(p, q):
    p = K.clip(p, K.epsilon(), 1)
    q = K.clip(q, K.epsilon(), 1)
    return K.sum(p * K.log(p / q), axis=-1)

In [ ]:
def kullback_leibler_div2(c):
    p = c[0]
    q = c[1]
    p = K.clip(p, K.epsilon(), 1)
    q = K.clip(q, K.epsilon(), 1)
    return K.sum(p * K.log(p / q), axis=-1)

In [ ]:
kvar = K.variable(np.array([[1, 2], [3, 4]]), dtype='float32')
K.eval(kvar)

In [ ]:
np_cooc = np.array([[0.06060606, 0.03030303, 0.75757576],
 [0.00456621, 0.00913242, 0.03652968],
 [0.11764706, 0.02941176, 0.05882353]])
np_logits = np.array([0.7, 0.15, 0.15])

fake_cooc = K.variable(np_cooc, dtype='float32')
fake_logits = K.variable(np_logits, dtype='float32')

In [ ]:
fake_logits.shape

In [ ]:
K.eval([kullback_leibler_div(fake_cooc[i], fake_logits) for i in range(3)])

In [ ]:
broad_logits = tf.tile(tf.expand_dims(fake_logits, 0), [3, 1])
broad_logits.shape
# out = tf.map_fn(kullback_leibler_div2, (fake_cooc, broad_logits), dtype=(tf.float32, tf.float32))

In [ ]:
K.eval(out)

In [ ]:
K.eval(tf.concat([fake_cooc, broad_logits], axis=0))

In [ ]:
K.eval(tf.tile(tf.expand_dims(fake_logits, 0), [3, 1]))

In [ ]:
K.eval(out)

In [ ]:
matrix = tf.keras.layers.Input(shape=(3,3))
logits = tf.keras.layers.Input(shape=(3,))
print(logits.shape)
print(matrix.shape)

# broad_logits = tf.tile(tf.expand_dims(logits, 0), [3, 1])
broad_logits = tf.tile(logits, [3, 1])
print(broad_logits.shape)
out = tf.map_fn(kullback_leibler_div2, (matrix, broad_logits), dtype=(tf.float32, tf.float32))

test_model = tensorflow.keras.models.Model(inputs=[input1, input2], outputs=out)